# Plotting histograms

Create histograms based on year and month of observations, as well as histograms based on the values.

The acceptable values (for instance between 0 and 450 µmol/l for oxygen) are set in the `config.jl` file. 

In [16]:
using Pkg
Pkg.activate("./")
using Glob
using Dates
using DIVAnd
using NCDatasets
using GeoDatasets
using CairoMakie, GeoMakie
using JupyterFormatter
enable_autoformat()
include("./config.jl")
monthlist = [Dates.monthname(mm) for mm = 1:12];

  Activating project at `~/Projects/EMODnet/EMODnet-Chemistry-GriddedMaps/src`
[ Info: Precompiling DIVAnd [efc8151c-67de-5a8f-9a35-d8f54746ae9d] (cache misses: wrong dep version loaded (6), incompatible header (2))


## All regions together

In [3]:
for varname in varlist
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    obsyears = Int64[]
    obsmonths = Int64[]

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsvalue, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)
        append!(obsyears, Dates.year.(obstime))
        append!(obsmonths, Dates.month.(obstime))
    end

    @info(extrema(obsyears))
    yearmin = minimum(obsyears)
    yearmax = maximum(obsyears)
    nobs_month = [length(findall(obsmonths .== mm)) for mm = 1:12]
    nobs_year = [length(findall(obsyears .== yyyy)) for yyyy = yearmin:yearmax]

    # Make figure
    fig1 = make_histogram(yearmin, yearmax, nobs_year, nobs_month, varname_)
    save(joinpath(figdir, "histogram_$varname.png"), fig1)
end

LoadError: UndefVarError: `varlist` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## One color by region

In [4]:
yearmin = 1960
yearmax = 2024

for varname in varlist
    @info("Working on variable $(varname)")
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    nyears_region = zeros(length(datafilelist), yearmax - yearmin + 1)
    nmonths_region = zeros(length(datafilelist), 12)

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsvalue, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)

        obsmonths = Dates.month.(obstime)
        obsyears = Dates.year.(obstime)
        nmonths_region[iii, :] = [length(findall(obsmonths .== mm)) for mm = 1:12]
        nyears_region[iii, :] =
            [length(findall(obsyears .== yyyy)) for yyyy = yearmin:yearmax]
    end

    fig = Figure(size = (1500, 500))
    ax = Axis(
        fig[1, 1],
        title = "Number of $(varname) observations per year",
        xticks = 1960:10:2020,
    )

    ax2 = Axis(
        fig[1, 2],
        xticks = 1:12,
        xtickformat = x -> Dates.monthname.(Int.(x)),
        xticklabelrotation = pi / 6,
        title = "Number of $(varname) observations per month",
    )

    bplist = []
    for jjj = 6:-1:1
        bp = barplot!(
            ax,
            yearmin:yearmax,
            dropdims(sum(nyears_region[1:jjj, :], dims = 1), dims = 1),
            strokewidth = 0.5,
            strokecolor = (:black, 0.5),
            color = collect(values(domaincolors))[jjj],
            label = collect(keys(domaincolors))[jjj],
        )
        push!(bplist, bp)
        barplot!(
            ax2,
            1:12,
            dropdims(sum(nmonths_region[1:jjj, :], dims = 1), dims = 1),
            strokewidth = 0.5,
            strokecolor = (:black, 0.5),
            color = collect(values(domaincolors))[jjj],
        )
        hidespines!(ax2, :t, :r)
    end
    axislegend(ax, reverse(bplist), collect(keys(domaincolors)), position = :lt)
    xlims!(ax, 1960 - 0.5, 2024 + 0.5)
    xlims!(ax2, 0.5, 12 + 0.5)
    hidespines!(ax, :t, :r)
    hidespines!(ax2, :t, :r)

    save(joinpath(figdir, "stacked_histogram_$varname.png"), fig)
end

LoadError: UndefVarError: `varlist` not defined in `Main`
Suggestion: check for spelling errors or missing imports.

## By values (all together)

In [38]:
nbins = 21
for varname in varlist
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    obsvalues = Float64[]

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsval, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)
        append!(obsvalues, obsval)
    end

    fig = Figure(size = (700, 400))
    ax = Axis(
        fig[1, 1],
        title = "$(varname_) histogram",
        xlabel = "Concentration ($(varunits[varname]))",
        ylabel = "Number of\nobservations\nper bin",
        ylabelrotation = 0,
    )
    hist!(
        ax,
        obsvalues,
        bins = collect(range(varrange[varname][1], varrange[varname][2], nbins)),
        strokewidth = 0.5,
        strokecolor = (:black, 0.5),
        color = :grey,
    )
    hidespines!(ax, :t, :r)
    # display(fig)
    save(joinpath(figdir, "histogram_value_$(varname).png"), fig)
end

## By value, with color

In [42]:
nbins = 21
for varname in varlist
    varname_ = replace(varname, "_" => " ")
    datafilelist = sort(Glob.glob("*$(varname)*.nc", datadir))

    obsvalues = Float64[]
    ndata = zeros(length(datafilelist), nbins - 1)
    bins = collect(range(varrange[varname][1], varrange[varname][2], nbins))

    for (iii, datafile) in enumerate(datafilelist)
        @debug("Reading data from $(basename(datafile))")
        obsval, obslon, obslat, obsdepth, obstime, obsids =
            loadobs(Float64, datafile, varname)
        append!(obsvalues, obsval)

        for jjj = 1:nbins-1
            ndata[iii, jjj] = sum((obsval .>= bins[jjj]) .& (obsval .< bins[jjj+1]))
        end

    end

    fig = Figure(size = (700, 400))
    ax = Axis(
        fig[1, 1],
        title = "$(varname_) histogram",
        xlabel = "Concentration ($(varunits[varname]))",
        ylabel = "Number of\nobservations\nper bin",
        ylabelrotation = 0,
    )

    bplist = []
    for kkk = 6:-1:1
        bp = barplot!(
            ax,
            bins[1:end-1] .+ 0.5,
            dropdims(sum(ndata[1:kkk, :], dims = 1), dims = 1),
            strokewidth = 0.5,
            strokecolor = (:black, 0.5),
            color = collect(values(domaincolors))[kkk],
            label = collect(keys(domaincolors))[kkk],
        )
        push!(bplist, bp)
    end

    axislegend(ax, reverse(bplist), collect(keys(domaincolors)), position = :rt)
    hidespines!(ax, :t, :r)
    # display(fig)
    save(joinpath(figdir, "histogram_value_$(varname).png"), fig)
end